In [1]:
%matplotlib notebook

In [2]:
import numpy as np
import json

In [3]:
from dask.distributed import Client, LocalCluster

client = Client(LocalCluster(n_workers=1, threads_per_worker=3))

In [4]:
smoldyn_template = template = """random_seed {seed} 
define K_A {k_a} 
define K_D {k_d} 
define MU {mu} 
define KAPPA {kappa} 
define GAMMA {gamma} 
define OUTPUT stdout 

dim 3 
boundaries x -8 8 
boundaries y -8 8 
boundaries z -8 8 
time_start 0 
time_stop {time_stop} 
time_step {time_step} 

species Gf Gb RNA P 
difc RNA {diffusion} 
difc P {diffusion} 
display_size all(all) 0.02 
color P(all) lightblue 
color RNA(all) navy 
color Gf(all) scarlet 
color Gb(all) darkred 

graphics opengl_good 
frame_thickness 0 

start_surface membrane 
action both all reflect 
color both blue 
thickness 0.01 
panel sphere 0 0 0 {cell_radius} 30 30 
end_surface 

start_surface nucleus 
action front RNA reflect 
action back P reflect 
color both red 
thickness 0.01 
panel sphere 0 0 0 {nucleus_radius} 30 30 
end_surface 

start_compartment inside 
surface nucleus 
point 0 0 0\n end_compartment 

start_compartment cytoplasm 
surface membrane 
point 0 0 0\n compartment andnot inside 
end_compartment 

reaction transcription Gf -> Gf + RNA MU 
reaction compartment=cytoplasm translation RNA -> RNA + P KAPPA 
reaction binding Gf + P <-> Gb K_A K_D 
reaction compartment=cytoplasm degradation RNA|P -> 0 GAMMA 
reaction compartment=inside degradation2 RNA|P -> 0 GAMMA 
{initial_state} 

output_files OUTPUT 
cmd B molcountheader OUTPUT 
cmd N {freq_point} molcount OUTPUT 
cmd A echo OUTPUT "--Simulation ends--\\n" 
cmd A listmols OUTPUT 

end_file 
"""

In [5]:
import smoldyn, CBM, WMM

base_parameters = {
    'k_a': 0.00166,
    'k_d': 0.1,
    'mu': 3.0,
    'kappa': 1.0,
    'gamma': 0.04,
    'diffusion': 0.6,
    'time_step': 0.1,
    'cell_radius': 6.0,
    'nucleus_radius': 2.5,
}


INIT_STOP = 1000
TSTOP = 1200
NTRAJ = 32

param_cd = {(c, d) for c in np.linspace(-8, 4, num=15) for d in np.linspace(-8, 4, num=15)}

In [ ]:
init_model = smoldyn.Cell(template, base_parameters)
# Generate initial state
initial_state = [smoldyn.Mol("Gf", 0.0, 0.0, 0.0)]
_, initial_state = init_model.run(initial_state, time_stop=INIT_STOP, n_trajectories=1, n_points=2)

In [ ]:
# Smoldyn
import time

smoldyn_data = []
wallclock_time = []
start = time.time()
for c, d in param_cd:
    smoldyn_parameters = {**base_parameters}
    smoldyn_parameters['mu'] *= 2**c
    smoldyn_parameters['kappa'] *= 2**c
    smoldyn_parameters['gamma'] *= 2**c
    smoldyn_parameters['diffusion'] *= 2**d
    smoldyn_parameters['model'] = 'smoldyn'
        
    model = smoldyn.Cell(template, smoldyn_parameters)
        
    history, time_data = model.run_threshold(
        initial_state, time_stop=TSTOP, n_trajectories=NTRAJ, n_points=500, threshold=0.05)

        
    smoldyn_data.append((smoldyn_parameters, history))
    wallclock_time.append((smoldyn_parameters, time_data))
    
    print("({}, {}): {}".format(c, d, time.time() - start))
    start = time.time()
    
with open('data/smoldyn_data.json', 'w') as f:
    json.dump(smoldyn_data, f)
    
with open('data/smoldyn_time.json', 'w') as f:
    json.dump(wallclock_time, f)

In [ ]:
cbm_base_parameters = {**base_parameters}
#cbm_base_parameters['model'] = 'CBM'
    
import reactionrates as rr
kentry, kexit = rr.entry_exit_rates(
    cbm_base_parameters['nucleus_radius'],
    cbm_base_parameters['cell_radius'],
    cbm_base_parameters['diffusion']) 
    
sigma = 0.01
V_nucleus = 4*np.pi*cbm_base_parameters['nucleus_radius']**3/3
k_a_cbmodel, k_d_cbmodel = rr.well_mixed_rates(
    cbm_base_parameters['k_a'],
    cbm_base_parameters['k_d'],
    sigma, 2*cbm_base_parameters['diffusion'],
    V_nucleus)
    
cbm_base_parameters['k_a'] = k_a_cbmodel
cbm_base_parameters['k_d'] = k_d_cbmodel
cbm_base_parameters['k_cn'] = kentry
cbm_base_parameters['k_nc'] = kexit

init_model = CBM.Cell(cbm_base_parameters)
# Generate initial state
initial_state = {'Gf': 1}
_, initial_state = init_model.run(initial_state, time_stop=INIT_STOP, n_trajectories=1, n_points=2)

In [ ]:
# CBM
import time

cbm_data = []
wallclock_time = []

for c, d in param_cd:
    cbm_parameters = {**base_parameters}
    cbm_parameters['mu'] *= 2**c
    cbm_parameters['kappa'] *= 2**c
    cbm_parameters['gamma'] *= 2**c
    cbm_parameters['diffusion'] *= 2**d
    #cbm_parameters['model'] = 'CBM'
    
    import reactionrates as rr
    kentry, kexit = rr.entry_exit_rates(
        cbm_parameters['nucleus_radius'],
        cbm_parameters['cell_radius'],
        cbm_parameters['diffusion']) 
    
    sigma = 0.01
    V_nucleus = 4*np.pi*cbm_parameters['nucleus_radius']**3/3
    k_a_cbmodel, k_d_cbmodel = rr.well_mixed_rates(
        cbm_parameters['k_a'],
        cbm_parameters['k_d'],
        sigma, 2*cbm_parameters['diffusion'],
        V_nucleus)
    
    cbm_parameters['k_a'] = k_a_cbmodel
    cbm_parameters['k_d'] = k_d_cbmodel
    cbm_parameters['k_cn'] = kentry
    cbm_parameters['k_nc'] = kexit
    
    model = CBM.Cell(cbm_parameters)
    
    start = time.time()
    history = model.run(initial_state, time_stop=TSTOP, n_trajectories=NTRAJ, n_points=500)
    end = time.time()
    
    cbm_data.append((cbm_parameters, history))
    wallclock_time.append((cbm_parameters, end - start))
    print("({}, {}): {}".format(c, d, end - start))
    
with open('data/cbm_data.json', 'w') as f:
    json.dump(cbm_data, f)
    
with open('data/cbm_time.json', 'w') as f:
    json.dump(wallclock_time, f)

In [6]:
import reactionrates as rr

wmm_base_parameters = {**base_parameters}
#wmm_base_parameters['model'] = 'WMM'
    
sigma = 0.01
V_nucleus = 4*np.pi*wmm_base_parameters['nucleus_radius']**3/3
k_a_wmmodel, k_d_wmmodel = rr.well_mixed_rates(
    wmm_base_parameters['k_a'],
    wmm_base_parameters['k_d'],
    sigma, 2*wmm_base_parameters['diffusion'],
    V_nucleus)
    
wmm_base_parameters['k_a'] = k_a_wmmodel
wmm_base_parameters['k_d'] = k_d_wmmodel


init_model = WMM.Cell(wmm_base_parameters)
# Generate initial state
initial_state = {'Gf': 1}
_, initial_state = init_model.run(initial_state, time_stop=INIT_STOP, n_trajectories=1, n_points=2)

In [7]:
# WMM
import time
import reactionrates as rr


wmm_data = []
wallclock_time = []

for c, d in param_cd:
    wmm_parameters = {**base_parameters}
    wmm_parameters['mu'] *= 2**c
    wmm_parameters['kappa'] *= 2**c
    wmm_parameters['gamma'] *= 2**c
    wmm_parameters['diffusion'] *= 2**d
    #wmm_parameters['model'] = 'WMM'
    
    sigma = 0.01
    V_nucleus = 4*np.pi*wmm_parameters['nucleus_radius']**3/3
    k_a_wmmodel, k_d_wmmodel = rr.well_mixed_rates(
        wmm_parameters['k_a'],
        wmm_parameters['k_d'],
        sigma, 2*wmm_parameters['diffusion'],
        V_nucleus)
    
    wmm_parameters['k_a'] = k_a_wmmodel
    wmm_parameters['k_d'] = k_d_wmmodel
    
    model = WMM.Cell(wmm_parameters)
    
    start = time.time()
    history = model.run(initial_state, time_stop=TSTOP, n_trajectories=NTRAJ, n_points=500)
    end = time.time()
    
    wmm_data.append((wmm_parameters, history))
    wallclock_time.append((wmm_parameters, end - start))
    print("({}, {}): {}".format(c, d, end - start))
    
with open('data/wmm_data.json', 'w') as f:
    json.dump(wmm_data, f)
    
with open('data/wmm_time.json', 'w') as f:
    json.dump(wallclock_time, f)

(4.0, -3.7142857142857144): 209.6192820072174
(-7.142857142857143, -3.7142857142857144): 0.10826802253723145
(-2.8571428571428577, -0.2857142857142865): 1.891035795211792
(-2.8571428571428577, 0.5714285714285712): 1.868027687072754
(-8.0, -4.571428571428571): 0.07950901985168457
(-2.0, -0.2857142857142865): 3.487325668334961
(-2.0, 0.5714285714285712): 3.396897077560425
(2.2857142857142847, -4.571428571428571): 64.12822318077087
(1.4285714285714288, -5.428571428571429): 36.64593005180359
(-1.1428571428571432, -8.0): 6.3014976978302
(-8.0, 1.4285714285714288): 0.07282066345214844
(-3.7142857142857144, -7.142857142857143): 1.05283522605896
(-2.8571428571428577, -7.142857142857143): 1.8718738555908203
(-2.0, -7.142857142857143): 3.35154128074646
(-4.571428571428571, -8.0): 0.5874700546264648
(4.0, -0.2857142857142865): 206.782621383667
(4.0, 0.5714285714285712): 210.2513198852539
(-5.428571428571429, 1.4285714285714288): 0.38028740882873535
(-6.285714285714286, -5.428571428571429): 0.2112

(-1.1428571428571432, -6.285714285714286): 6.331234455108643
(1.4285714285714288, 4.0): 37.72902750968933
(-8.0, 3.1428571428571423): 0.06717777252197266
(-2.8571428571428577, -1.1428571428571432): 1.9122555255889893
(-2.0, -1.1428571428571432): 3.5007150173187256
(2.2857142857142847, 3.1428571428571423): 69.67804098129272
(-7.142857142857143, 2.2857142857142847): 0.11397194862365723
(-4.571428571428571, -6.285714285714286): 0.9696142673492432
(4.0, 2.2857142857142847): 214.47264528274536
(-5.428571428571429, 3.1428571428571423): 0.5330040454864502
(-6.285714285714286, -7.142857142857143): 0.3161015510559082
(-5.428571428571429, -3.7142857142857144): 0.4003000259399414
(-4.571428571428571, 4.0): 0.6375730037689209
(-7.142857142857143, -2.8571428571428577): 0.1244974136352539
(-7.142857142857143, -2.0): 0.1021735668182373
(4.0, -4.571428571428571): 207.30391693115234
(-2.8571428571428577, 3.1428571428571423): 1.9382789134979248
(-2.0, 3.1428571428571423): 3.4286048412323
(-8.0, -5.42857